# Laboratorio 9 - Deep Learning

### Autores

- Angel Higueros 20460
- Fredy Velasquez 201011

## Task 1 - Práctica

### 1. Preparación de datos

In [1]:
# Importa las bibliotecas necesarias
import pandas as pd
import numpy as np

In [2]:
# Carga los datos CSV en DataFrames de pandas
train_df = pd.read_csv('demand-forecasting-kernels-only/train.csv')
test_df = pd.read_csv('demand-forecasting-kernels-only/test.csv')
sample_submission_df = pd.read_csv('demand-forecasting-kernels-only/sample_submission.csv')

In [3]:
# Muestra las columnas de cada DataFrame
print("Columnas en train_df:")
print(train_df.columns)

print("\nColumnas en test_df:")
print(test_df.columns)

print("\nColumnas en sample_submission_df:")
print(sample_submission_df.columns)

Columnas en train_df:
Index(['date', 'store', 'item', 'sales'], dtype='object')

Columnas en test_df:
Index(['id', 'date', 'store', 'item'], dtype='object')

Columnas en sample_submission_df:
Index(['id', 'sales'], dtype='object')


In [4]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [5]:
from sklearn.preprocessing import StandardScaler


# Selecciona las columnas relevantes para la normalización en 'train_df'
columnas_numericas = ['sales']

# Crea un objeto StandardScaler
scaler = StandardScaler()

# Ajusta el escalador en los datos de entrenamiento y transforma los datos
train_df[columnas_numericas] = scaler.fit_transform(train_df[columnas_numericas])

# Verifica los datos transformados en 'train_df'
print("Datos de entrenamiento normalizados:")
print(train_df.head())


Datos de entrenamiento normalizados:
         date  store  item     sales
0  2013-01-01      1     1 -1.362804
1  2013-01-02      1     1 -1.432246
2  2013-01-03      1     1 -1.328083
3  2013-01-04      1     1 -1.362804
4  2013-01-05      1     1 -1.466966


In [ ]:
# Preprocesamiento de Datos

# División de series temporales

# Supongamos que tienes una columna de fecha llamada "date" en tus datos.
# Ordena los datos por fecha para asegurarte de que los más recientes estén en el conjunto de prueba.
train_df['date'] = pd.to_datetime(train_df['date'])
train_df.sort_values(by='date', inplace=True)

# Define la fecha de corte para separar los datos en entrenamiento, validación y prueba.
fecha_corte_validacion = '2022-07-01'
fecha_corte_prueba = '2022-10-01'

# Conjunto de entrenamiento
train_set = train_df[train_df['date'] < fecha_corte_validacion]

# Conjunto de validación
validation_set = train_df[(train_df['date'] >= fecha_corte_validacion) & (train_df['date'] < fecha_corte_prueba)]

# Conjunto de prueba
test_set = train_df[train_df['date'] >= fecha_corte_prueba]

# Generación de secuencias

# Define la ventana de tiempo histórico que deseas utilizar para predecir los próximos 3 meses.
ventana_historica = 90  # Por ejemplo, utiliza los últimos 90 días para predecir 3 meses.

def crear_secuencias(df, ventana_historica):
    secuencias = []
    objetivos = []
    for i in range(len(df) - ventana_historica):
        secuencia = df['sales'].values[i:i+ventana_historica]
        objetivo = df['sales'].values[i+ventana_historica:i+ventana_historica+90]  # Pronóstico de 3 meses
        secuencias.append(secuencia)
        objetivos.append(objetivo)
    return np.array(secuencias), np.array(objetivos)

# Crea secuencias para los conjuntos de entrenamiento, validación y prueba
x_train, y_train = crear_secuencias(train_set, ventana_historica)
x_validation, y_validation = crear_secuencias(validation_set, ventana_historica)
x_test, y_test = crear_secuencias(test_set, ventana_historica)

# Verifica las dimensiones de los conjuntos de datos
print("Dimensiones de x_train:", x_train.shape)
print("Dimensiones de y_train:", y_train.shape)
print("Dimensiones de x_validation:", x_validation.shape)
print("Dimensiones de y_validation:", y_validation.shape)
print("Dimensiones de x_test:", x_test.shape)
print("Dimensiones de y_test:", y_test.shape)

## Task 2 - Teoria 

1. ¿Cuál es el problema del gradiente de fuga en las redes LSTM y cómo afecta la efectividad de LSTM para el pronóstico de series temporales?

El problema del gradiente de fuga en las redes LSTM se refiere a la tendencia de los gradientes durante la retropropagación a volverse muy pequeños o muy grandes a medida que se propagan hacia atrás en el tiempo a través de muchas unidades de tiempo en una secuencia larga. Esto puede dificultar el entrenamiento efectivo de modelos LSTM para series temporales, ya que los gradientes pequeños pueden llevar a un entrenamiento lento o a un estancamiento, mientras que los gradientes grandes pueden provocar explosiones del gradiente. Esto afecta la efectividad de LSTM para pronósticos de series temporales, ya que puede dificultar la captura de relaciones a largo plazo en los datos.

2. ¿Cómo se aborda la estacionalidad en los datos de series temporales cuando se utilizan LSTM para realizar pronósticos y qué papel juega la diferenciación en el proceso?

La estacionalidad en los datos de series temporales se aborda en LSTM mediante la diferenciación. La diferenciación implica calcular la diferencia entre valores sucesivos en la serie temporal para eliminar la tendencia y la estacionalidad. Esto convierte la serie en una serie estacionaria que es más fácil de modelar. La diferenciación ayuda a que LSTM capture mejor los patrones temporales en los datos y mejore la precisión de los pronósticos.

3. ¿Cuál es el concepto de "tamaño de ventana" en el pronóstico de series temporales con LSTM y cómo afecta la elección del tamaño de ventana a la capacidad del modelo para capturar patrones a corto y largo plazo?

El "tamaño de ventana" en el pronóstico de series temporales con LSTM se refiere a la cantidad de pasos de tiempo anteriores que se utilizan como entrada para predecir el siguiente paso de tiempo. Un tamaño de ventana más grande permite al modelo capturar patrones a largo plazo en los datos, pero también puede hacer que el modelo sea menos sensible a patrones a corto plazo. Por otro lado, un tamaño de ventana más pequeño se centra en patrones a corto plazo, pero puede perder patrones a largo plazo. La elección del tamaño de ventana depende de la naturaleza de los datos y del equilibrio deseado entre la captura de patrones a corto y largo plazo.